In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
import random
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

2023-11-02 20:16:00.653198: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

#Read dataset
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

#add location to each sample
train_a["location"] = "A"
train_b["location"] = "B"
train_c["location"] = "C"
X_train_estimated_a["location"] = "A"
X_train_estimated_b["location"] = "B"
X_train_estimated_c["location"] = "C"
X_train_observed_a["location"] = "A"
X_train_observed_b["location"] = "B"
X_train_observed_c["location"] = "C"

#remove extra minute 00 sample
X_train_observed_a = X_train_observed_a.iloc[:-1,:]
X_train_observed_b = X_train_observed_b.iloc[:-1,:]
X_train_observed_c = X_train_observed_c.iloc[:-1,:]

#add date_calc column same as date_forecast column to observed data
X_train_observed_a.insert(0, "date_calc", X_train_observed_a["date_forecast"])
X_train_observed_b.insert(0, "date_calc", X_train_observed_b["date_forecast"])
X_train_observed_c.insert(0, "date_calc", X_train_observed_c["date_forecast"])

#concat all the samples and remove date_calc column
X_train_raw = pd.concat([X_train_observed_a,
                     X_train_observed_b,
                     X_train_observed_c,
                     X_train_estimated_a,
                     X_train_estimated_b,
                     X_train_estimated_c])

#feature indicating time between date_calc and date_forecast
X_train_raw["calc_time"] =(X_train_raw["date_forecast"] - X_train_raw["date_calc"]).astype('timedelta64[s]')

#fill nans
X_train_raw["snow_density:kgm3"] = X_train_raw["snow_density:kgm3"].apply(
    lambda a : np.isnan(a)
    ).map({True: 0, False: 1})
X_train_raw["ceiling_height_agl:m"] = X_train_raw["ceiling_height_agl:m"].apply(
    lambda a : -1000 if np.isnan(a) else a
)
X_train_raw["cloud_base_agl:m"] = X_train_raw["ceiling_height_agl:m"].apply(
    lambda a : -1000 if np.isnan(a) else a
)

X_train_raw["date_forecast"] = X_train_raw["date_forecast"] + pd.Timedelta(hours=-2)

#create seperate dataframes for measurments at minute 00, 15, 30 and 45
X_train00 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 0)].reset_index().iloc[:,1:]
X_train15 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 15)].reset_index().iloc[:,1:]
X_train30 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 30)].reset_index().iloc[:,1:]
X_train45 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 45)].reset_index().iloc[:,1:]

#remove redundant data
X_train15 = X_train15.iloc[:,2:-2]
X_train30 = X_train30.iloc[:,2:-2]
X_train45 = X_train45.iloc[:,2:-2]

#join observations into single sample
X_train = X_train00.join(X_train15, lsuffix="_00", rsuffix="_15").join(X_train30.join(X_train45, lsuffix="_30", rsuffix="_45"))

#rename column for merging with targets
X_train = X_train.rename(columns={"date_forecast" : "time"})

#concat target values and drop NaN values
targets = pd.concat([train_a,
                     train_b,
                     train_c]).dropna()

#merge weatherfeatures with corresponding target pv measurement
dataset = pd.merge(X_train, targets, how="inner", on=["time", "location"])

#shuffle dataset
dataset = dataset.sample(frac=1, random_state=43).reset_index().iloc[:,1:]

#split into features and targets
datasetX = dataset.iloc[:, :-1]
datasetY = dataset.iloc[:, -1:]

#add day_of_year and hour feature columns
datasetX["day"] = datasetX["time"].dt.day_of_year
datasetX["hour"] = datasetX["time"].dt.hour

#get indexes of samples in the months of the test dataset
indexMayJuneJuly = datasetX["time"].apply(lambda time : time.month in [5, 6, 7])

#OHE encoding for catagorical feature "location"
datasetX["location_A"] = datasetX["location"].apply(lambda a : a == "A").map({True: 1, False: 0})
datasetX["location_B"] = datasetX["location"].apply(lambda a : a == "B").map({True: 1, False: 0})
datasetX["location_C"] = datasetX["location"].apply(lambda a : a == "C").map({True: 1, False: 0})

datasetX = datasetX.drop("location", axis=1)

#move datecalc column
#date_calc_column = datasetX.pop("date_calc")
#datasetX.insert(183, "is_not_calculated", date_calc_column) #!constant 183 can be source of bugs

#fix nans for some reason???
#datasetX["is_not_calculated"] = datasetX["is_not_calculated"].fillna(method="ffill")

#map location labels to numbers
#datasetX["location"] = datasetX["location"].map({"A": 0, "B": 1, "C": 2})

#drop time and date_calc columns
datasetX = datasetX.iloc[:,2:]

#calculate mean and std for normalizing data, values should also be used for normalizing test data
dataMean = datasetX.mean()
dataStd = datasetX.std()

#normalize data
datasetX.iloc[:,:-4] = ((datasetX.iloc[:,:-4]-dataMean[:-4])/dataStd[:-4]).fillna(value=0)


#partition into training and evalset
trainsetX = datasetX.iloc[:,:]
trainsetY = datasetY.iloc[:,:]
trainsetIndexMayJuneJuly = indexMayJuneJuly[:]
evalsetX = datasetX.iloc[85000:,:]
evalsetY = datasetY.iloc[85000:,:]
evalsetIndexMayJuneJuly = indexMayJuneJuly[85000:]

display(datasetX)

/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69482/1200425358.py:121: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  datasetX.iloc[:,:-4] = ((datasetX.iloc[:,:-4]-dataMean[:-4])/dataStd[:-4]).fillna(value=0)


,absolute_humidity_2m:gm3_00,air_density_2m:kgm3_00,ceiling_height_agl:m_00,clear_sky_energy_1h:J_00,clear_sky_rad:W_00,cloud_base_agl:m_00,dew_or_rime:idx_00,dew_point_2m:K_00,diffuse_rad:W_00,diffuse_rad_1h:J_00,direct_rad:W_00,direct_rad_1h:J_00,effective_cloud_cover:p_00,elevation:m_00,fresh_snow_12h:cm_00,fresh_snow_1h:cm_00,fresh_snow_24h:cm_00,fresh_snow_3h:cm_00,fresh_snow_6h:cm_00,is_day:idx_00,is_in_shadow:idx_00,msl_pressure:hPa_00,precip_5min:mm_00,precip_type_5min:idx_00,pressure_100m:hPa_00,pressure_50m:hPa_00,prob_rime:p_00,rain_water:kgm2_00,relative_humidity_1000hPa:p_00,sfc_pressure:hPa_00,snow_density:kgm3_00,snow_depth:cm_00,snow_drift:idx_00,snow_melt_10min:mm_00,snow_water:kgm2_00,sun_azimuth:d_00,sun_elevation:d_00,super_cooled_liquid_water:kgm2_00,t_1000hPa:K_00,total_cloud_cover:p_00,visibility:m_00,wind_speed_10m:ms_00,wind_speed_u_10m:ms_00,wind_speed_v_10m:ms_00,wind_speed_w_1000hPa:ms_00,calc_time,absolute_humidity_2m:gm3_15,air_density_2m:kgm3_15,ceiling_height_agl:m_15,clear_sky_energy_1h:J_15,clear_sky_rad:W_15,cloud_base_agl:m_15,dew_or_rime:idx_15,dew_point_2m:K_15,diffuse_rad:W_15,diffuse_rad_1h:J_15,direct_rad:W_15,direct_rad_1h:J_15,effective_cloud_cover:p_15,elevation:m_15,fresh_snow_12h:cm_15,fresh_snow_1h:cm_15,fresh_snow_24h:cm_15,fresh_snow_3h:cm_15,fresh_snow_6h:cm_15,is_day:idx_15,is_in_shadow:idx_15,msl_pressure:hPa_15,precip_5min:mm_15,precip_type_5min:idx_15,pressure_100m:hPa_15,pressure_50m:hPa_15,prob_rime:p_15,rain_water:kgm2_15,relative_humidity_1000hPa:p_15,sfc_pressure:hPa_15,snow_density:kgm3_15,snow_depth:cm_15,snow_drift:idx_15,snow_melt_10min:mm_15,snow_water:kgm2_15,sun_azimuth:d_15,sun_elevation:d_15,super_cooled_liquid_water:kgm2_15,t_1000hPa:K_15,total_cloud_cover:p_15,visibility:m_15,wind_speed_10m:ms_15,wind_speed_u_10m:ms_15,wind_speed_v_10m:ms_15,wind_speed_w_1000hPa:ms_15,absolute_humidity_2m:gm3_30,air_density_2m:kgm3_30,ceiling_height_agl:m_30,clear_sky_energy_1h:J_30,clear_sky_rad:W_30,cloud_base_agl:m_30,dew_or_rime:idx_30,dew_point_2m:K_30,diffuse_rad:W_30,diffuse_rad_1h:J_30,direct_rad:W_30,direct_rad_1h:J_30,effective_cloud_cover:p_30,elevation:m_30,fresh_snow_12h:cm_30,fresh_snow_1h:cm_30,fresh_snow_24h:cm_30,fresh_snow_3h:cm_30,fresh_snow_6h:cm_30,is_day:idx_30,is_in_shadow:idx_30,msl_pressure:hPa_30,precip_5min:mm_30,precip_type_5min:idx_30,pressure_100m:hPa_30,pressure_50m:hPa_30,prob_rime:p_30,rain_water:kgm2_30,relative_humidity_1000hPa:p_30,sfc_pressure:hPa_30,snow_density:kgm3_30,snow_depth:cm_30,snow_drift:idx_30,snow_melt_10min:mm_30,snow_water:kgm2_30,sun_azimuth:d_30,sun_elevation:d_30,super_cooled_liquid_water:kgm2_30,t_1000hPa:K_30,total_cloud_cover:p_30,visibility:m_30,wind_speed_10m:ms_30,wind_speed_u_10m:ms_30,wind_speed_v_10m:ms_30,wind_speed_w_1000hPa:ms_30,absolute_humidity_2m:gm3_45,air_density_2m:kgm3_45,ceiling_height_agl:m_45,clear_sky_energy_1h:J_45,clear_sky_rad:W_45,cloud_base_agl:m_45,dew_or_rime:idx_45,dew_point_2m:K_45,diffuse_rad:W_45,diffuse_rad_1h:J_45,direct_rad:W_45,direct_rad_1h:J_45,effective_cloud_cover:p_45,elevation:m_45,fresh_snow_12h:cm_45,fresh_snow_1h:cm_45,fresh_snow_24h:cm_45,fresh_snow_3h:cm_45,fresh_snow_6h:cm_45,is_day:idx_45,is_in_shadow:idx_45,msl_pressure:hPa_45,precip_5min:mm_45,precip_type_5min:idx_45,pressure_100m:hPa_45,pressure_50m:hPa_45,prob_rime:p_45,rain_water:kgm2_45,relative_humidity_1000hPa:p_45,sfc_pressure:hPa_45,snow_density:kgm3_45,snow_depth:cm_45,snow_drift:idx_45,snow_melt_10min:mm_45,snow_water:kgm2_45,sun_azimuth:d_45,sun_elevation:d_45,super_cooled_liquid_water:kgm2_45,t_1000hPa:K_45,total_cloud_cover:p_45,visibility:m_45,wind_speed_10m:ms_45,wind_speed_u_10m:ms_45,wind_speed_v_10m:ms_45,wind_speed_w_1000hPa:ms_45,day,hour,location_A,location_B,location_C
0,0.030517,0.452377,-0.436209,-0.627831,-0.626218,-0.436209,-0.028478,0.199491,-0.651186,-0.658520,-0.444494,-0.449894,0.941231,-0.558794,-0.149069,-0.085624,-0.188846,-0.103516,-0.120872,-0.967054,0.876565,0.649192,-0.169291,-0.21

In [3]:
numModels = 20

models = []

i = 0
while(i < numModels):
  if(i < 10):
    models.append(tf.keras.models.Sequential([
          #tf.keras.layers.GaussianNoise(stddev=0.1, seed=42),
          tf.keras.layers.Dense(100, activation="tanh"),
          tf.keras.layers.Dense(100, activation="relu"),
          tf.keras.layers.Dense(1, activation="relu"),
      ]))
  else:
    models.append(tf.keras.models.Sequential([
          #tf.keras.layers.GaussianNoise(stddev=0.1, seed=42),
          tf.keras.layers.Dense(150, activation="tanh"),
          tf.keras.layers.Dense(120, activation="relu"),
          tf.keras.layers.Dense(90, activation="relu"),
          tf.keras.layers.Dense(60, activation="relu"),
          tf.keras.layers.Dense(1, activation="relu"),
      ]))
  models[i].compile(
      optimizer=tf.keras.optimizers.experimental.Adadelta(learning_rate=1,
                                                          #weight_decay=0.0001
                                                          ),
      loss="mean_absolute_error"
  )

  history = models[i].fit(x = trainsetX.sample(frac=0.5 if i < 10 else 1, random_state=i),
                      y= trainsetY.sample(frac=0.5 if i < 10 else 1, random_state=i),
                      batch_size = 1000,
                      epochs=20,
                      verbose=0
                  )

  #retry if model is nonsensical
  loss = models[i].evaluate(evalsetX, evalsetY)
  if(loss > 200):
      del models[i]
      continue

  print(i, " complete")
  i+=1


249/249 [==============================] - 1s 2ms/step - loss: 83.2724
0  complete
249/249 [==============================] - 1s 2ms/step - loss: 82.7396
1  complete
249/249 [==============================] - 1s 2ms/step - loss: 86.6911
2  complete
249/249 [==============================] - 1s 2ms/step - loss: 83.2898
3  complete
249/249 [==============================] - 1s 2ms/step - loss: 82.8882
4  complete
249/249 [==============================] - 0s 2ms/step - loss: 84.4677
5  complete
249/249 [==============================] - 1s 2ms/step - loss: 84.3747
6  complete
249/249 [==============================] - 1s 2ms/step - loss: 83.7478
7  complete
249/249 [==============================] - 1s 2ms/step - loss: 82.8540
8  complete
249/249 [==============================] - 1s 2ms/step - loss: 83.0477
9  complete
249/249 [==============================] - 1s 2ms/step - loss: 74.9850
10  complete
249/249 [==============================] - 1s 3ms/step - loss: 75.1823
11  complete
24

In [69]:
#!ReWrite
def evaluate_models(models, X, Y):
    preds = X.iloc[:,1:2]

    for i in range(len(models)):
        preds[str(i)] = models[i].predict(X)
        
    
    preds = preds.iloc[:,1:]
    
    
    preds["final"] = preds.mean(axis=1)
    preds["losses"] = (preds["final"] - Y["pv_measurement"]).apply(lambda a : np.abs(a))
    return preds["losses"].mean()


In [70]:

print("entire:")
results = evaluate_models(models ,evalsetX, evalsetY)
print(results)

print("may june july:")
resultsMayJuneJuly = evaluate_models(models ,evalsetX[evalsetIndexMayJuneJuly], evalsetY[evalsetIndexMayJuneJuly])
print(resultsMayJuneJuly)

print("location A")
resultA = evaluate_models(models ,evalsetX[evalsetX["location_A"].apply(lambda a : a == 1)], 
                         evalsetY[evalsetX["location_A"].apply(lambda a : a == 1)])
print(resultA)

print("location B")
resultB = evaluate_models(models ,evalsetX[evalsetX["location_B"].apply(lambda a : a == 1)], 
                         evalsetY[evalsetX["location_B"].apply(lambda a : a == 1)])
print(resultB)

print("location C")
resultC = evaluate_models(models ,evalsetX[evalsetX["location_C"].apply(lambda a : a == 1)], 
                         evalsetY[evalsetX["location_C"].apply(lambda a : a == 1)])
print(resultC)

entire:
251/251 [==============================] - 1s 2ms/step
71.59717768999626
may june july:
52/52 [==============================] - 0s 2ms/step
152.4090214616747
location A
90/90 [==============================] - 0s 3ms/step
152.58090833097805
location B
90/90 [==============================] - 0s 2ms/step
33.23559459736758
location C
72/72 [==============================] - 0s 2ms/step
17.623983273711286


In [4]:
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

X_test_raw = pd.concat([
                     X_test_estimated_a,
                     X_test_estimated_b,
                     X_test_estimated_c])

#feature indicating time between date_calc and date_forecast
X_test_raw["calc_time"] =(X_test_raw["date_forecast"] - X_test_raw["date_calc"]).astype('timedelta64[s]')

#fill nans
X_test_raw["snow_density:kgm3"] = X_test_raw["snow_density:kgm3"].apply(
    lambda a : np.isnan(a)
    ).map({True: 0, False: 1})
X_test_raw["ceiling_height_agl:m"] = X_test_raw["ceiling_height_agl:m"].apply(
    lambda a : -1000 if np.isnan(a) else a
)
X_test_raw["cloud_base_agl:m"] = X_test_raw["ceiling_height_agl:m"].apply(
    lambda a : -1000 if np.isnan(a) else a
)

X_test_raw["date_forecast"] = X_test_raw["date_forecast"] + pd.Timedelta(hours=-2)

#create seperate dataframes for measurments at minute 00, 15, 30 and 45
X_test00 = X_test_raw[X_test_raw["date_forecast"].apply(lambda time: time.minute == 0)].reset_index().iloc[:,1:]
X_test15 = X_test_raw[X_test_raw["date_forecast"].apply(lambda time: time.minute == 15)].reset_index().iloc[:,1:]
X_test30 = X_test_raw[X_test_raw["date_forecast"].apply(lambda time: time.minute == 30)].reset_index().iloc[:,1:]
X_test45 = X_test_raw[X_test_raw["date_forecast"].apply(lambda time: time.minute == 45)].reset_index().iloc[:,1:]

#remove redundant data
X_test15 = X_test15.iloc[:,2:-2]
X_test30 = X_test30.iloc[:,2:-2]
X_test45 = X_test45.iloc[:,2:-2]

#join observations into single sample
X_test_estimated = X_test00.join(X_test15, lsuffix="_00", rsuffix="_15").join(X_test30.join(X_test45, lsuffix="_30", rsuffix="_45"))


X_test_estimated = X_test_estimated.rename(columns={"date_forecast" : "time"})

parse_dates = ['time']
X_test_targets = pd.read_csv("test.csv", parse_dates=parse_dates)

X_test = pd.merge(X_test_estimated, X_test_targets, on=["time", "location"], how="right").iloc[:,:-2]


X_test["day"] = X_test["time"].dt.day_of_year
X_test["hour"] = X_test["time"].dt.hour

X_test["location_A"] = X_test["location"].apply(lambda a : a == "A").map({True: 1, False: 0})
X_test["location_B"] = X_test["location"].apply(lambda a : a == "B").map({True: 1, False: 0})
X_test["location_C"] = X_test["location"].apply(lambda a : a == "C").map({True: 1, False: 0})

X_test = X_test.drop("location", axis=1)

X_test = X_test.iloc[:,2:]


X_test.iloc[:,:-4] = ((X_test.iloc[:,:-4]-dataMean[:-4])/dataStd[:-4]).fillna(value=0)

display(X_test)



/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_59706/1064356002.py:63: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  X_test.iloc[:,:-4] = ((X_test.iloc[:,:-4]-dataMean[:-4])/dataStd[:-4]).fillna(value=0)


,absolute_humidity_2m:gm3_00,air_density_2m:kgm3_00,ceiling_height_agl:m_00,clear_sky_energy_1h:J_00,clear_sky_rad:W_00,cloud_base_agl:m_00,dew_or_rime:idx_00,dew_point_2m:K_00,diffuse_rad:W_00,diffuse_rad_1h:J_00,direct_rad:W_00,direct_rad_1h:J_00,effective_cloud_cover:p_00,elevation:m_00,fresh_snow_12h:cm_00,fresh_snow_1h:cm_00,fresh_snow_24h:cm_00,fresh_snow_3h:cm_00,fresh_snow_6h:cm_00,is_day:idx_00,is_in_shadow:idx_00,msl_pressure:hPa_00,precip_5min:mm_00,precip_type_5min:idx_00,pressure_100m:hPa_00,pressure_50m:hPa_00,prob_rime:p_00,rain_water:kgm2_00,relative_humidity_1000hPa:p_00,sfc_pressure:hPa_00,snow_density:kgm3_00,snow_depth:cm_00,snow_drift:idx_00,snow_melt_10min:mm_00,snow_water:kgm2_00,sun_azimuth:d_00,sun_elevation:d_00,super_cooled_liquid_water:kgm2_00,t_1000hPa:K_00,total_cloud_cover:p_00,visibility:m_00,wind_speed_10m:ms_00,wind_speed_u_10m:ms_00,wind_speed_v_10m:ms_00,wind_speed_w_1000hPa:ms_00,calc_time,absolute_humidity_2m:gm3_15,air_density_2m:kgm3_15,ceiling_height_agl:m_15,clear_sky_energy_1h:J_15,clear_sky_rad:W_15,cloud_base_agl:m_15,dew_or_rime:idx_15,dew_point_2m:K_15,diffuse_rad:W_15,diffuse_rad_1h:J_15,direct_rad:W_15,direct_rad_1h:J_15,effective_cloud_cover:p_15,elevation:m_15,fresh_snow_12h:cm_15,fresh_snow_1h:cm_15,fresh_snow_24h:cm_15,fresh_snow_3h:cm_15,fresh_snow_6h:cm_15,is_day:idx_15,is_in_shadow:idx_15,msl_pressure:hPa_15,precip_5min:mm_15,precip_type_5min:idx_15,pressure_100m:hPa_15,pressure_50m:hPa_15,prob_rime:p_15,rain_water:kgm2_15,relative_humidity_1000hPa:p_15,sfc_pressure:hPa_15,snow_density:kgm3_15,snow_depth:cm_15,snow_drift:idx_15,snow_melt_10min:mm_15,snow_water:kgm2_15,sun_azimuth:d_15,sun_elevation:d_15,super_cooled_liquid_water:kgm2_15,t_1000hPa:K_15,total_cloud_cover:p_15,visibility:m_15,wind_speed_10m:ms_15,wind_speed_u_10m:ms_15,wind_speed_v_10m:ms_15,wind_speed_w_1000hPa:ms_15,absolute_humidity_2m:gm3_30,air_density_2m:kgm3_30,ceiling_height_agl:m_30,clear_sky_energy_1h:J_30,clear_sky_rad:W_30,cloud_base_agl:m_30,dew_or_rime:idx_30,dew_point_2m:K_30,diffuse_rad:W_30,diffuse_rad_1h:J_30,direct_rad:W_30,direct_rad_1h:J_30,effective_cloud_cover:p_30,elevation:m_30,fresh_snow_12h:cm_30,fresh_snow_1h:cm_30,fresh_snow_24h:cm_30,fresh_snow_3h:cm_30,fresh_snow_6h:cm_30,is_day:idx_30,is_in_shadow:idx_30,msl_pressure:hPa_30,precip_5min:mm_30,precip_type_5min:idx_30,pressure_100m:hPa_30,pressure_50m:hPa_30,prob_rime:p_30,rain_water:kgm2_30,relative_humidity_1000hPa:p_30,sfc_pressure:hPa_30,snow_density:kgm3_30,snow_depth:cm_30,snow_drift:idx_30,snow_melt_10min:mm_30,snow_water:kgm2_30,sun_azimuth:d_30,sun_elevation:d_30,super_cooled_liquid_water:kgm2_30,t_1000hPa:K_30,total_cloud_cover:p_30,visibility:m_30,wind_speed_10m:ms_30,wind_speed_u_10m:ms_30,wind_speed_v_10m:ms_30,wind_speed_w_1000hPa:ms_30,absolute_humidity_2m:gm3_45,air_density_2m:kgm3_45,ceiling_height_agl:m_45,clear_sky_energy_1h:J_45,clear_sky_rad:W_45,cloud_base_agl:m_45,dew_or_rime:idx_45,dew_point_2m:K_45,diffuse_rad:W_45,diffuse_rad_1h:J_45,direct_rad:W_45,direct_rad_1h:J_45,effective_cloud_cover:p_45,elevation:m_45,fresh_snow_12h:cm_45,fresh_snow_1h:cm_45,fresh_snow_24h:cm_45,fresh_snow_3h:cm_45,fresh_snow_6h:cm_45,is_day:idx_45,is_in_shadow:idx_45,msl_pressure:hPa_45,precip_5min:mm_45,precip_type_5min:idx_45,pressure_100m:hPa_45,pressure_50m:hPa_45,prob_rime:p_45,rain_water:kgm2_45,relative_humidity_1000hPa:p_45,sfc_pressure:hPa_45,snow_density:kgm3_45,snow_depth:cm_45,snow_drift:idx_45,snow_melt_10min:mm_45,snow_water:kgm2_45,sun_azimuth:d_45,sun_elevation:d_45,super_cooled_liquid_water:kgm2_45,t_1000hPa:K_45,total_cloud_cover:p_45,visibility:m_45,wind_speed_10m:ms_45,wind_speed_u_10m:ms_45,wind_speed_v_10m:ms_45,wind_speed_w_1000hPa:ms_45,day,hour,location_A,location_B,location_C
0,-0.595904,0.834926,-0.382669,-0.627834,-0.626244,-0.382669,-0.028554,-0.517632,-0.651235,-0.658524,-0.444504,-0.449899,0.390542,-0.685740,-0.148871,-0.085506,-0.188709,-0.103329,-0.120629,-0.967213,0.876756,0.335965,-0.169307,-0.

In [5]:
#!ReWrite
def models_predict(models, X):
    preds = X.iloc[:,1:2]

    for i in range(len(models)):
        preds[str(i)] = models[i].predict(X)

    preds = preds.iloc[:,1:]
    preds["final"] = preds.mean(axis=1)
    return preds["final"]

In [6]:
test_preds = pd.read_csv("sample_submission.csv")
test_preds["prediction"] = models_predict(models, X_test)
display(test_preds)
test_preds.to_csv("preds/EnsambleOfEnsamblesFixAlignTest1.csv", index=False)

68/68 [==============================] - 0s 2ms/step


,id,prediction
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,40.099602
4,4,404.075958
...,...,...
2155,2155,45.132843
2156,2156,17.548037
2157,2157,0.499489
2158,2158,0.000000
